In [ ]:
import pandas as pd
import numpy as np

from IPython.display import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import EfficientNetB4, Xception, ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input
from keras.layers import Dense, Conv2D, Flatten,Input,Dropout,MaxPooling2D
from tensorflow.keras import datasets, layers, models
from tensorflow.math import confusion_matrix
from matplotlib.pyplot import imread
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.image import ImageDataGenerator
import os
import cv2
import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.models import Sequential
import tensorflow_hub as hub


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

In [ ]:


if tf.test.is_gpu_available():
    print("GPU is available.")
else:
    print("GPU is not available.")


In [ ]:
labels = pd.read_csv("../input/dog-breed-identification/labels.csv")
labels


In [ ]:
sample_submission_csv=pd.read_csv("/kaggle/input/dog-breed-identification/sample_submission.csv")
sample_submission_csv

In [ ]:
labelnames=pd.read_csv("/kaggle/input/dog-breed-identification/sample_submission.csv").keys()[1:]
labelnames


In [ ]:
# datamın olduğu klasörün yolunu bir değişkene atıyorum.
my_data_dir = "/kaggle/input/dog-breed-identification"

In [ ]:

os.listdir(my_data_dir)

In [ ]:


test_path = os.path.join(my_data_dir, 'test')     
train_path = os.path.join(my_data_dir, 'train')  

# os.path.join bizim işletim sistemize göre dosya yollarını belirliyor.

In [ ]:
test=os.listdir(test_path)


In [ ]:

test[0]

In [ ]:
len(test) 

In [ ]:
test = np.array(sorted(test))
test

In [ ]:
train=os.listdir(train_path)


In [ ]:
len(train) 

In [ ]:
train=np.array(sorted(train))
train

'U', Unicode tipini temsil eder.Unicode, dunya genelindeki bilgisayar
 sistemlerinde metin karakterlerini temsil etmek için kullanılan bir standarttır.  
36, her bir karakter dizisinin en fazla 36 karakter uzunluğunda olduğunu belirtir.
         

In [ ]:
# Create pathnames from image ID's
filenames = [train_path + fname + ".jpg" for fname in labels["id"]]

# Check the first 10 filenames
filenames[:10]

In [ ]:
import os
if len(os.listdir(train_path)) == len(filenames):
  print("Filenames match actual amount of files!")
else:
  print("Filenames do not match actual amount of files, check the target directory.")

In [ ]:
#dog-breed-identification dizininde gezinin ve dosya sayısını listeleyin:klasorlerin yapısını gormek için 
for dirpath, dirnames, filenames in os.walk(my_data_dir):
    print(
        f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

In [ ]:
labels["filenames"] = filenames
labels

In [ ]:
labels["breed"].value_counts()#.plot.bar(figsize=(20, 10));

In [ ]:
path1 = os.path.join(train_path,train[0])
path2 = os.path.join(test_path,test[0])
train_img = imread(path1)
test_img = imread(path2)
# elimizdeki resimler png formatında biz bunları imread ile array'e çeviriyoruz.
#çunku imshow metodu  array alabildiği için imread bu işlevi görüyor  
#imread'i matplotlib içerisinden import ettik
train_img

"train_img" adlı bir array verisidir. Bu array, bir resmin piksel değerlerini içerir. Her bir piksel, üç değerden oluşan bir RGB (Kırmızı, Yeşil, Mavi) renk değeri içerir.

Array'in şekli (boyutu) 3 boyutludur. İlk boyut, resmin yüksekliğini temsil eder. İkinci boyut, resmin genişliğini temsil eder. Üçüncü boyut ise RGB renk kanallarını temsil eder.

Örneğin, "train_img" array'inin şekli (yükseklik, genişlik, 3) olabilir. Bu durumda, resim 3 renk kanalına (RGB) sahip olur. Her bir piksel, bu üç renk kanalının değerlerini içeren bir vektör olarak temsil edilir.

Örneğin, "train_img" array'inin ilk pikseli şu şekilde temsil edilebilir: [108, 46, 0]. Bu pikselin kırmızı bileşeni 108, yeşil bileşeni 46 ve mavi bileşeni 0'dır.

Bu şekilde, tüm resmin piksel değerleri array içinde temsil edilir.



In [ ]:
train_img.shape

In [ ]:
# imshow ile array formatındaki resmi artık okutabiliriz.
plt.imshow(train_img)
plt.show()

In [ ]:
# View an image
import random


def view_random_image(target_dir, target_class):
    """Display and return an image given the target_dir and target_class.
    Args:
        target_dir (str): The full or relative path of the directory of the target.
        target_class (str): The name of the target_class you want to display and return.
    Returns:
        img: (np.ndarray): Image as numpy array.
    """
    # Setup target directory (we'll view images from here)
    target_folder = os.path.join(target_dir, target_class)

    # Get a random image path
    random_image = random.sample(os.listdir(target_folder), 1)

    # Read in the image and plot it using matplotlib
    img = imread(os.path.join(target_folder, random_image[0]))
    plt.imshow(img)
    plt.title(target_class)
    plt.axis("off")

    print(f"Image shape: {img.shape}")  # show the shape of the image

    return img


img = view_random_image(target_dir=my_data_dir, target_class="train")

In [ ]:
x = [
    imread(os.path.join(my_data_dir, "train", image)).shape[0]
    for image in os.listdir(os.path.join(my_data_dir, "train"))
]
z = [
    imread(os.path.join(my_data_dir, "train", image)).shape[1]
    for image in os.listdir(os.path.join(my_data_dir, "train"))
]

In [ ]:
np.mean(x), np.median(x)

In [ ]:
np.mean(z), np.median(z)

In [ ]:
sns.scatterplot(x=x, y=z)
plt.show()

In [ ]:
labels["breed"].value_counts()#.plot.bar(figsize=(20, 10));

In [ ]:
codes = range(len(labelnames))
breed_to_code = dict(zip(labelnames, codes))
code_to_breed = dict(zip(codes, labelnames))
labels['target'] =  [breed_to_code[x] for x in labels["breed"]]
labels

In [ ]:
train_path = '/kaggle/input/dog-breed-identification/train'
test_path =  "/kaggle/input/dog-breed-identification/test"

In [ ]:
import cv2
import os
import numpy as np

# Dosya yollarını içeren dataframe'i oluşturun (df)
# Burada df, verilerinizi içeren bir dataframe olmalıdır.
def read_and_resize_image_rgb(file_path):
    img = cv2.imread(file_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    img_rgb_resized = cv2.resize(img_rgb, (224, 224))
    return img_rgb_resized

# Dosya yollarını içeren sütunu kullanarak resimleri okuyun ve boyutlandırın
processed_images = []
for idx, filename in enumerate(labels['id']):
    img_path = os.path.join(train_path, filename + '.jpg')
    resized_img = read_and_resize_image_rgb(img_path)
    processed_images.append(resized_img)


# Liste içindeki resimleri NumPy array'e dönüştürün
processed_images_array = np.array(processed_images)




In [ ]:
print("processed_images shape:", processed_images_array.shape)
print("processed_images dtype:", processed_images_array.dtype)
print("labels shape:",labels['target'].shape)  # Bu sadece numpy dizileri için geçerlidir
print("labels dtype:",labels['target'].dtype)

# processed_images'in ilk resmi görselleştirme
import matplotlib.pyplot as plt
plt.imshow(processed_images_array[0])
plt.show()

In [ ]:
def read_and_resize_image_rgb(file_path):
    img = cv2.imread(file_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    img_rgb_resized = cv2.resize(img_rgb, (224, 224))
    return img_rgb_resized

# Dosya yollarını içeren sütunu kullanarak resimleri okuyun ve boyutlandırın
processed2_images = []
for idx, filename in enumerate(sample_submission_csv['id']):
    img_path = os.path.join(test_path, filename + '.jpg')
    resized_img = read_and_resize_image_rgb(img_path)
    processed2_images.append(resized_img)


# Liste içindeki resimleri NumPy array'e dönüştürün
processed2_images_array = np.array(processed2_images)

In [ ]:
print("processed_images shape:", processed2_images_array.shape)
print("processed_images dtype:", processed2_images_array.dtype)
print("labels shape:",sample_submission_csv['id'].shape)  # Bu sadece numpy dizileri için geçerlidir
print("labels dtype:",sample_submission_csv['id'].dtype)

# processed_images'in ilk resmi görselleştirme
import matplotlib.pyplot as plt
plt.imshow(processed2_images_array[0])
plt.show()

# Train Test Split

In [ ]:
X=processed_images_array
y=labels["target"]



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# Scaling

Let's check the images if they are needed to be scaled or not

In [ ]:
train_img.max(),train_img.min()

In [ ]:
test_img.max(),test_img.min()

As we see above, images need to be scaled

# Image Data Generator

Image Manipulation

We can use the ImageDataGenerator to manipulate the images with rotation, resizing, and scaling so the model becomes more robust to different images that our data set doesn't have. ImageDataGenerator does the followings.

Accepts a batch of images used for training.
Applies a series of random transformations to each image in the batch.
Replaces the original batch with randomly transformed batch.
Training the CNN on this randomly transformed batch.
The goal of applying data augmentation is to have a more generalized model.

Data augmentation is a way to try and prevent a model overfitting. If your model is overfiting (e.g. the validation loss keeps increasing), you may want to try using data augmentation.

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen=ImageDataGenerator(rescale=1./255)          

In [ ]:
training_set=train_datagen.flow(X_train,y=y_train,batch_size=128)
testing_set=test_datagen.flow(X_test,y=y_test,batch_size=128)

In [ ]:
model = Sequential([
    Input(shape=(224, 224, 3)),
    Conv2D(16, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='sigmoid'),
    Dropout(0.2),
    Dense(120, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Early Stopping ayarları
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
model.summary()

In [ ]:
history=model.fit_generator(training_set,validation_data = testing_set,epochs = 5,   callbacks=[early_stopping] )

In [ ]:
# Eğitim sürecinin sonunda modelin başarı oranını değerlendirin
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Eğitim geçmişini DataFrame'e dönüştürme
history_df = pd.DataFrame(history.history)

# Grafik oluşturma
plt.figure(figsize=(12, 6))

# Loss değerlerini göster
plt.subplot(1, 2, 1)
plt.plot(history_df['loss'], label='loss')
plt.plot(history_df['val_loss'], label='val_loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Accuracy değerlerini göster
plt.subplot(1, 2, 2)
plt.plot(history_df['accuracy'], label='accuracy')
plt.plot(history_df['val_accuracy'], label='val_accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


# Transfer Learning (EfficientNetB4, Xception, ResNet50)

In [ ]:
#Bu fonksiyon, önceden eğitilmiş bir modelin belirli bir katmanından özellik haritaları almak için kullanılır. 
def get_features(model_name, model_preprocessor, input_size, data):
    input_layer = Input(input_size)
    preprocessor = model_preprocessor(input_layer)
    base_model = model_name(weights='imagenet', include_top=False, input_shape=input_size)(preprocessor)
    avg = GlobalAveragePooling2D()(base_model)
    feature_extractor = Model(inputs=input_layer, outputs=avg)
    feature_maps = feature_extractor.predict(data, verbose=1)
    print('Feature maps shape:', feature_maps.shape)
    return feature_maps

In [ ]:
##Bu örnekte, get_features fonksiyonu içinde, belirli bir modeli ve preprocessorü kullanarak özellik çıkarımı yapıyoruz.
##Daha sonra, örnek modeller (EfficientNetB4, Xception, ResNet50) ve preprocessorleri 
#(effnet_preprocessor, xception_preprocessor, resnet_preprocessor) kullanarak bu özellik çıkarımını gerçekleştiriyoruz.

effnet_preprocessor = tf.keras.applications.efficientnet.preprocess_input
effnet_features = get_features(EfficientNetB4, effnet_preprocessor, (224, 224, 3),X)

xception_preprocessor = tf.keras.applications.xception.preprocess_input
xception_features = get_features(Xception, xception_preprocessor, (224, 224, 3), X)

resnet_preprocessor = tf.keras.applications.resnet50.preprocess_input
resnet_features = get_features(ResNet50, resnet_preprocessor, (224, 224, 3), X)

In [ ]:
final_features = np.concatenate([effnet_features, xception_features, resnet_features], axis=-1)
print('Final feature maps shape:', final_features.shape)


In [ ]:
from keras.layers import BatchNormalization


model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(final_features.shape[1],)))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
#model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(120, activation='softmax'))


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [ ]:
#checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

In [ ]:
history = model.fit(final_features, y,
                    batch_size=128,
                    epochs=20,
                    validation_split=0.2,
                    callbacks=[ early_stopping])

In [ ]:
history_df = pd.DataFrame(history.history)
history_df[10:]

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from kerastuner.tuners import RandomSearch

# Verilerinizi yükleyin ve işleyin
# Örnek olarak, veri yükleme işlemlerini eklemelisiniz.

# Modeli oluşturun ve derleyin
def build_model(hp):
    model = Sequential()
    model.add(Dropout(0.7, input_shape=(final_features.shape[1],)))
    model.add(Dense(120, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

# Keras Tuner RandomSearch için tuner'ı oluşturun
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Uygun bir sayı seçebilirsiniz
    directory='my_dir',
    project_name='dog_breed_classification'
)



# Hiperparametre aramasını başlatın
tuner.search(x=final_features, y=y, epochs=20, validation_split=0.2,)

# En iyi modeli ve hiperparametreleri alın
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values


In [ ]:
best_hyperparameters

In [ ]:
history_df = pd.DataFrame(history.history)
history_df[15:]

In [ ]:

# Eğitim geçmişini DataFrame'e dönüştürme
history_df = pd.DataFrame(history.history)

# Grafik oluşturma
plt.figure(figsize=(12, 6))

# Loss değerlerini göster
plt.subplot(1, 2, 1)
plt.plot(history_df['loss'], label='loss')
plt.plot(history_df['val_loss'], label='val_loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Accuracy değerlerini göster
plt.subplot(1, 2, 2)
plt.plot(history_df['accuracy'], label='accuracy')
plt.plot(history_df['val_accuracy'], label='val_accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


# Model Evaluation

In [ ]:
#best_model = load_model('best_model.h5')

In [ ]:
effnet_preprocessor = tf.keras.applications.efficientnet.preprocess_input
effnet_features = get_features(EfficientNetB4, effnet_preprocessor, (224, 224, 3),processed2_images_array)

xception_preprocessor = tf.keras.applications.xception.preprocess_input

xception_features = get_features(Xception, xception_preprocessor, (224, 224, 3), processed2_images_array)

resnet_preprocessor = tf.keras.applications.resnet50.preprocess_input
resnet_features = get_features(ResNet50, resnet_preprocessor, (224, 224, 3), processed2_images_array)

In [ ]:
final_features2 = np.concatenate([effnet_features, xception_features, resnet_features], axis=-1)
print('Final feature maps shape:', final_features2.shape)

In [ ]:
#Predict test labels given test data features.
y_pred = model.predict(final_features2, batch_size=128)

In [ ]:
dog_breds=sorted(labelnames)


n_classes = len(dog_breeds)
class_to_num = dict(zip(dog_breeds, range(n_classes)))
class_to_num

In [ ]:
for b in dog_breds:
    sample_submission_csv[b] = y_pred[:,class_to_num[b]]
sample_submission_csv.to_csv('pred.csv', index=None)